In [64]:
from pandas.io import data, wb
import datetime
import os.path
import os
import math
import pandas as pd
import numpy




In [65]:
df_train = pd.read_csv('data/spy-ohlc.csv', index_col='Date', parse_dates=True)
if (not os.path.isfile('data/spy-ohlc.csv')):
    os.mknod('data/spy-ohlc.csv')
df_dividends = pd.read_csv('data/spy-dividends.csv', index_col='Date', parse_dates=True)
df_train = df_train.join(df_dividends)
df_train['dividend'].fillna(0, inplace=True)

In [66]:
pma10 = df_train['Close'].rolling(window=10,center=False).mean() + df_train['dividend'].rolling(window=10, center=False).mean()
pma50 = df_train['Close'].rolling(window=50,center=False).mean()+ df_train['dividend'].rolling(window=50, center=False).mean()
vma50 = df_train['Volume'].rolling(window=50, center=False).mean()



In [67]:
#distance from MAs
diffma10  = (df_train['Close']/pma10 -1)*100
diffma50 = (df_train['Close']/pma50 -1)*100

#tangential of ma lines
#pma10tan = (pma10[1:]/pma10[:-1].values-1)*100
#pma50tan = (pma50[1:]/pma50[:-1].values-1)*100

#sequence of higher high/higher low
rangex1 = (df_train['High']/df_train['Low']-1)*100
rangex2 = rangex1.shift(1)
rangex3 = rangex1.shift(2)
rangex4 = rangex1.shift(3)
rangex5 = rangex1.shift(4)

#rolling average of hh and hl
rangema10 = rangex1.rolling(window=10, center=False).mean()

nvx1 = df_train['Volume']/vma50


In [68]:
#add features for training data

#distance from MAs
df_train = df_train.assign(diffma10=diffma10)
df_train = df_train.assign(diffma50=diffma50)

#tangential of ma lines
#df_train = df_train.assign(pma10tan=pma10tan)
#df_train = df_train.assign(pma20tan=pma20tan)
#df_train = df_train.assign(pma50tan=pma50tan)
#df_train = df_train.assign(pma200tan=pma200tan)

df_train = df_train.assign(rangex1=rangex1)
df_train = df_train.assign(rangex2=rangex2)
df_train = df_train.assign(rangex3=rangex3)
df_train = df_train.assign(rangex4=rangex4)
df_train = df_train.assign(rangex5=rangex5)

df_train = df_train.assign(rangema10=rangema10)


#last day volume
df_train = df_train.assign(nvx1=nvx1)

In [69]:
df_train.to_csv("data/spy-train.csv")

remove = ['Open','High','Low','Close','Volume','Adj Close','dividend']
df_train = df_train.drop(remove, axis=1)

In [70]:
def predict(X_train, y_train, X_test):  
    from sklearn.cross_validation import train_test_split
    from sklearn import cross_validation
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC


    #model = LogisticRegression(random_state=1)
    
    model = RandomForestRegressor(n_estimators=50)
    #model = GradientBoostingClassifier(n_estimators=100)
    #best so far
    #model = KNeighborsClassifier(n_neighbors=15)
    
    
    from sklearn import metrics
    
    #y_eval_comp = y_eval_pred[:,1] > 0.55
    #numpy.count_nonzero(y_eval_comp)
    
    #print("recall:", metrics.recall_score(y_eval, y_eval_comp, average='binary'))
    #print("precision:", metrics.precision_score(y_eval, y_eval_comp, average='binary'))
    #print("f1 score:", metrics.f1_score(y_eval, y_eval_comp, average='binary'))
    
    scores = cross_validation.cross_val_score(model, X_train, y_train, cv=5,scoring='median_absolute_error')
    print("score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
    
    model.fit(X_train, y_train)
    y_test_pred = model.predict(X_test);
    print(y_test_pred)
    return



In [58]:
df_train.describe()

,diffma10,diffma50,rangex1,rangex2,rangex3,rangex4,rangex5,rangema10,nvx1
count,5824.000000,5784.000000,5833.000000,5832.000000,5831.000000,5830.000000,5829.000000,5824.000000,5784.000000
mean,0.115908,0.661207,1.373216,1.373210,1.373348,1.373463,1.373587,1.374129,1.040135
std,1.775738,3.929484,1.040859,1.040948,1.040984,1.041036,1.041082,0.828040,0.529433
min,-15.974173,-27.180044,0.133601,0.133601,0.133601,0.133601,0.133601,0.395508,0.015866
25%,-0.752197,-1.173311,0.738036,0.738025,0.738122,0.738371,0.738859,0.838070,0.739618
50%,0.301936,1.139629,1.116308,1.116042,1.116308,1.116359,1.116410,1.161549,0.944151
75%,1.144987,3.132692,1.677041,1.677071,1.677101,1.677131,1.677162,1.655136,1.224181
max,8.571041,14.114149,12.673709,12.673709,12.673709,12.673709,12.673709,9.151972,12.385631


In [72]:
X_train = df_train.values[200:-10, :]
X_test = df_train.values[-10:, :]
print(X_train.shape, X_test.shape)
print(rangex1.shape, rangex1.shift(-1).shape)
y_train = rangex1.shift(-1).values[200:-10]
print(len(y_train))
predict(X_train, y_train, X_test)


(5623, 9) (10, 9)
(5833,) (5833,)
5623
Precision score: -0.35 (+/- 0.06)
[ 1.27513086  1.08066643  0.998271    1.1219203   1.28301797  1.10809284
  1.25848412  0.95562453  0.95432348  0.83956533]


In [167]:
def e_predict(X_train, y_train, X_test):
    from sklearn import cross_validation
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import GaussianNB
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.ensemble import VotingClassifier

    clf1 = LogisticRegression(random_state=1)
    clf2 = RandomForestClassifier(n_estimators=350)
    clf3 = GaussianNB()
    clf4 = GradientBoostingClassifier(n_estimators=100)
    clf5 = KNeighborsClassifier(n_neighbors=15)
    
    eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3), 
                                        ('gb', clf4), ('knn',clf5)], voting='soft')

    for clf in [clf1, clf2, clf3, clf4, clf5, eclf]: 
        print(type(clf).__name__, "\n")
        scores = cross_validation.cross_val_score(clf, X_train, y_train, cv=5, scoring='precision')
        print("Score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
        clf.fit(X_train, y_train)
        y_test_pred = clf.predict_proba(X_test);
        print(y_test_pred)


In [152]:
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression(random_state=1)
na = type(clf1).__name__
print(na)
    

LogisticRegression
